In [1]:
%matplotlib inline
import csv
import numpy as np
from scipy.signal import *
from numpy.fft import *
import matplotlib.pyplot as plt
from ipywidgets import * # Pour insérer des widgets (éléments graphiques) dans le notebook
# Attention! Installer scikit-learn pour avoir accès aux algorithmes de classification
# Installer version 0.20 au plus récente (0.22.1) pour avoir accès à la nouvelle fonction tree.plot_tree()
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier # Méthode KNN
from pandas import * # Structures de données
from sklearn.naive_bayes import GaussianNB
from scipy.ndimage import interpolation
from scipy.stats import entropy

In [2]:
feat_df = DataFrame(columns=['step type','Gyro0X mean', 'Gyro0Y mean', 'Gyro0Z mean',
                                         'Accel0X mean','Accel0Y mean','Accel0Z mean',
                                         'Gyro1X mean', 'Gyro1Y mean','Gyro1Z mean',
                                         'Accel1X mean', 'Accel1Y mean','Accel1Z mean',
                             
                                          'Gyro0X std', 'Gyro0Y std','Gyro0Z std',
                                         'Accel0X std', 'Accel0Y std', 'Accel0Z std',
                                         'Gyro1X std', 'Gyro1Y std', 'Gyro1Z std', 
                                         'Accel1X std', 'Accel1Y std', 'Accel1Z std',
                             
                                         'Gyro0X ent', 'Gyro0Y ent', 'Gyro0Z ent', 
                                         'Accel0X ent', 'Accel0Y ent', 'Accel0Z ent', 
                                         'Gyro1X ent', 'Gyro1Y ent','Gyro1Z ent',
                                         'Accel1X ent','Accel1Y ent','Accel1Z ent',
                             
                                         'Gyro0X corr', 'Gyro0Y corr', 'Gyro0Z corr', 
                                         'Accel0X corr', 'Accel0Y corr', 'Accel0Z corr', 
                                         'Gyro1X corr', 'Gyro1Y corr','Gyro1Z corr',
                                         'Accel1X corr','Accel1Y corr','Accel1Z corr'])

In [3]:
step_type_array = ["brush_back","brush_forward","dig","heel","step"]
size_array = [29,29,29,29,29]
ext = ".txt"
dancer = "Ilyas"

for i in range(len(size_array)):
    step_type = step_type_array[i]
    size = size_array[i]
    
    for k in range(1):
        if k == 0:
            path_prep =  "etapdance2" + "/" + dancer + "/" + step_type + "/" + step_type + "_cut_prep" + "/" + step_type + "_cut_prep_"
        elif k == 1:
            path_prep =  "etapdance2" + "/" + dancer + "/" + step_type + "/" + step_type + "_filt" + "/" + step_type + "_filt_"

        for n in range(size+1):
            step_type_nb = str(n)
            data_input = path_prep + step_type_nb + ext

            data_prep = pandas.read_csv(data_input,names=['Gyro0X (deg)','Gyro0Y (deg)','Gyro0Z (deg)','Accel0X (g)','Accel0Y (g)','Accel0Z (g)','Gyro1X (deg)','Gyro1Y (deg)','Gyro1Z (deg)','Accel1X (g)','Accel1Y (g)','Accel1Z (g)'], dtype='a') 
            
            # pour calcul corrélation
            step_corr = "step"
            data_input_corr = "etapdance2" + "/" + dancer + "/" + step_corr + "/" + step_corr + "_cut_prep_mean" + ext
            data_int_mean = pandas.read_csv(data_input_corr,names=['Gyro0X (deg)','Gyro0Y (deg)','Gyro0Z (deg)','Accel0X (g)','Accel0Y (g)','Accel0Z (g)','Gyro1X (deg)','Gyro1Y (deg)','Gyro1Z (deg)','Accel1X (g)','Accel1Y (g)','Accel1Z (g)'], dtype='a') 


            # MPU0 6050 (talon)
            Gyro0X = np.array(data_prep['Gyro0X (deg)'])[1:].astype(float)
            Gyro0Y = np.array(data_prep['Gyro0Y (deg)'])[1:].astype(float)
            Gyro0Z = np.array(data_prep['Gyro0Z (deg)'])[1:].astype(float)
            
            Accel0X = np.array(data_prep['Accel0X (g)'])[1:].astype(float)
            Accel0Y = np.array(data_prep['Accel0Y (g)'])[1:].astype(float)
            Accel0Z = np.array(data_prep['Accel0Z (g)'])[1:].astype(float)
            
            # geste moyen MPU0 6050 (talon) pour corrélation
            Gyro0X_int_mean = np.array(data_int_mean['Gyro0X (deg)'])[1:].astype(float)
            Gyro0Y_int_mean = np.array(data_int_mean['Gyro0Y (deg)'])[1:].astype(float)
            Gyro0Z_int_mean = np.array(data_int_mean['Gyro0Z (deg)'])[1:].astype(float)
            
            Accel0X_int_mean = np.array(data_int_mean['Accel0X (g)'])[1:].astype(float)
            Accel0Y_int_mean = np.array(data_int_mean['Accel0Y (g)'])[1:].astype(float)
            Accel0Z_int_mean = np.array(data_int_mean['Accel0Z (g)'])[1:].astype(float)


            # MPU1 6050 (pointe)
            Gyro1X = np.array(data_prep['Gyro1X (deg)'])[1:].astype(float)
            Gyro1Y = np.array(data_prep['Gyro1Y (deg)'])[1:].astype(float)
            Gyro1Z = np.array(data_prep['Gyro1Z (deg)'])[1:].astype(float)

            Accel1X = np.array(data_prep['Accel1X (g)'])[1:].astype(float)
            Accel1Y = np.array(data_prep['Accel1Y (g)'])[1:].astype(float)
            Accel1Z = np.array(data_prep['Accel1Z (g)'])[1:].astype(float)
            
            
            # geste moyen MPU1 6050 (pointe) pour corrélation
            Gyro1X_int_mean = np.array(data_int_mean['Gyro0X (deg)'])[1:].astype(float)
            Gyro1Y_int_mean = np.array(data_int_mean['Gyro0Y (deg)'])[1:].astype(float)
            Gyro1Z_int_mean = np.array(data_int_mean['Gyro0Z (deg)'])[1:].astype(float)
            
            Accel1X_int_mean = np.array(data_int_mean['Accel0X (g)'])[1:].astype(float)
            Accel1Y_int_mean = np.array(data_int_mean['Accel0Y (g)'])[1:].astype(float)
            Accel1Z_int_mean = np.array(data_int_mean['Accel0Z (g)'])[1:].astype(float)
            

            array_feat = [[step_type,   np.mean(Gyro0X),np.mean(Gyro0Y),np.mean(Gyro0Z),
                                        np.mean(Accel0X),np.mean(Accel0Y),np.mean(Accel0Z),
                                        np.mean(Gyro1X),np.mean(Gyro1Y),np.mean(Gyro1Z),
                                        np.mean(Accel1X),np.mean(Accel1Y),np.mean(Accel1Z),

                                        np.std(Gyro0X),np.std(Gyro0Y),np.std(Gyro0Z),
                                        np.std(Accel0X),np.std(Accel0Y),np.std(Accel0Z),
                                        np.std(Gyro1X),np.std(Gyro1Y),np.std(Gyro1Z),
                                        np.std(Accel1X),np.std(Accel1Y),np.std(Accel1Z),

                                        entropy(abs(Gyro0X)),entropy(abs(Gyro0Y)),entropy(abs(Gyro0Z)),
                                        entropy(abs(Accel0X)),entropy(abs(Accel0Y)),entropy(abs(Accel0Z)),
                                        entropy(abs(Gyro1X)),entropy(abs(Gyro1Y)),entropy(abs(Gyro1Z)),
                                        entropy(abs(Accel1X)),entropy(abs(Accel1Y)),entropy(abs(Accel1Z)),
                                        
                                       pandas.Series(Gyro0X_int_mean).corr(pandas.Series(Gyro0X)),
                                       pandas.Series(Gyro0Y_int_mean).corr(pandas.Series(Gyro0Y)),
                                       pandas.Series(Gyro0Z_int_mean).corr(pandas.Series(Gyro0Z)),
                                       pandas.Series(Accel0X_int_mean).corr(pandas.Series(Accel0X)),
                                       pandas.Series(Accel0Y_int_mean).corr(pandas.Series(Accel0Y)),
                                       pandas.Series(Accel0Z_int_mean).corr(pandas.Series(Accel0Z)),

                                       pandas.Series(Gyro1X_int_mean).corr(pandas.Series(Gyro1X)),
                                       pandas.Series(Gyro1Y_int_mean).corr(pandas.Series(Gyro1Y)),
                                       pandas.Series(Gyro1Z_int_mean).corr(pandas.Series(Gyro1Z)),
                                       pandas.Series(Accel1X_int_mean).corr(pandas.Series(Accel1X)),
                                       pandas.Series(Accel1Y_int_mean).corr(pandas.Series(Accel1Y)),
                                       pandas.Series(Accel1Z_int_mean).corr(pandas.Series(Accel1Z))]]


            feat_df_temp = DataFrame(array_feat, columns=['step type','Gyro0X mean', 'Gyro0Y mean', 'Gyro0Z mean',
                                                 'Accel0X mean','Accel0Y mean','Accel0Z mean',
                                                 'Gyro1X mean', 'Gyro1Y mean','Gyro1Z mean',
                                                 'Accel1X mean', 'Accel1Y mean','Accel1Z mean',

                                                 'Gyro0X std', 'Gyro0Y std','Gyro0Z std',
                                                 'Accel0X std', 'Accel0Y std', 'Accel0Z std',
                                                 'Gyro1X std', 'Gyro1Y std', 'Gyro1Z std', 
                                                 'Accel1X std', 'Accel1Y std', 'Accel1Z std',

                                                 'Gyro0X ent', 'Gyro0Y ent', 'Gyro0Z ent', 
                                                 'Accel0X ent', 'Accel0Y ent', 'Accel0Z ent', 
                                                 'Gyro1X ent', 'Gyro1Y ent','Gyro1Z ent',
                                                 'Accel1X ent','Accel1Y ent','Accel1Z ent',
                                                          
                                                 'Gyro0X corr', 'Gyro0Y corr', 'Gyro0Z corr', 
                                                 'Accel0X corr', 'Accel0Y corr', 'Accel0Z corr', 
                                                 'Gyro1X corr', 'Gyro1Y corr','Gyro1Z corr',
                                                 'Accel1X corr','Accel1Y corr','Accel1Z corr'])

            feat_df = feat_df.append(feat_df_temp,ignore_index =True, sort =False)

feat_df

,step type,Gyro0X mean,Gyro0Y mean,Gyro0Z mean,Accel0X mean,Accel0Y mean,Accel0Z mean,Gyro1X mean,Gyro1Y mean,Gyro1Z mean,...,Gyro0Z corr,Accel0X corr,Accel0Y corr,Accel0Z corr,Gyro1X corr,Gyro1Y corr,Gyro1Z corr,Accel1X corr,Accel1Y corr,Accel1Z corr
0,brush_back,-0.303090,0.445582,0.900953,-0.425362,-0.321025,-0.075076,0.093210,0.576390,-0.488588,...,0.021334,-0.896312,-0.586817,0.311237,-0.615592,-0.300841,-0.039491,0.820905,-0.806124,-0.207714
1,brush_back,-0.804103,0.778729,0.656035,-0.802740,-0.356401,0.527884,-0.218565,0.712612,-0.811779,...,-0.488536,0.288410,-0.689911,-0.367221,0.830491,0.548562,-0.461337,-0.132179,-0.480029,0.736755
2,brush_back,-0.483284,0.099523,0.671152,-0.895103,0.176660,0.250757,0.033120,0.660048,-0.657554,...,0.590808,0.491569,-0.777304,-0.594159,-0.241179,0.136326,-0.613078,-0.362299,-0.307279,0.437076
3,brush_back,-0.420795,0.857320,0.492942,-0.828882,0.214157,0.296114,0.300410,0.704306,-0.597177,...,-0.149079,0.776380,-0.742825,-0.264531,0.432922,0.312907,-0.194126,0.697073,-0.361166,0.448092
4,brush_back,-0.233294,0.780857,0.128108,-0.398938,0.198793,-0.084910,-0.062820,0.777260,-0.836095,...,-0.427841,-0.335706,0.866421,0.722346,-0.146748,0.342308,-0.397386,-0.354730,0.975954,-0.654217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,step,-0.825654,-0.792377,-0.382782,-0.797949,0.116887,-0.753207,-0.839907,-0.756880,-0.214080,...,-0.508153,-0.376871,-0.526737,-0.212757,0.903982,-0.789534,-0.467012,0.566276,0.606204,-0.053802
146,step,-0.719378,-0.644035,-0.416058,-0.526402,-0.559097,-0.409226,-0.819481,-0.699651,-0.305164,...,0.466091,-0.583395,0.221291,-0.773961,0.808710,-0.109794,0.566370,0.164469,-0.684357,0.773386
147,step,0.217472,0.176696,0.120290,0.009514,0.151321,-0.173656,0.236381,0.118894,0.148566,...,0.875047,0.380563,0.983205,0.672377,0.468232,0.696862,-0.238383,-0.385173,0.980861,-0.697894
148,step,0.144586,0.202612,0.342466,0.046507,-0.141659,0.177900,0.011131,0.193060,0.433581,...,0.770707,0.460420,-0.985181,-0.524108,0.035338,0.625288,0.611701,0.413239,0.960697,-0.859167


In [ ]:
pandas.set_option('display.max_rows', None)
feat_df

In [4]:
feat_df.to_csv("etapdance2/dataset_cut_ilyas_math_features.txt",index=False)

In [ ]:
feat_df[:[:2]]